In [42]:
import numpy as np
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import date
import datapackage

import yfinance as yf
from yahoofinancials import YahooFinancials

In [43]:
###########################################Pull SP500 list from datahub#################################
data_url = 'https://datahub.io/core/s-and-p-500-companies-financials/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])

#data.info()

In [44]:
col_list=['Symbol', 'Name', 'Sector', 'Market Cap']
sp_list=data[col_list]
sp_list['Market Cap']=sp_list['Market Cap'].div(1000000000)

sp_list.to_csv('./data/sp_list.csv', index=False)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
 "########################################Loop in each ticker's history#########################\n",
start='1990-01-01'
end='2020-11-06'
stock_list=sp_list['Symbol'].tolist()
yahoo_financials=YahooFinancials(stock_list)
data=yahoo_financials.get_historical_price_data(start_date=start, end_date=end, time_interval='weekly')

#dict_items = data.items()
#first_two = list(dict_items)[:2]
#print(first_two)

In [46]:
stock_database=pd.DataFrame({a: {x['formatted_date']: x['adjclose'] for x in data[a]['prices']} for a in stock_list if 'prices' in data[a]})
stock_database.head()

,MMM,AOS,ABT,ABBV,ACN,ATVI,AYI,ADBE,AAP,AMD,...,WYNN,XEL,XRX,XLNX,XL,XYL,YUM,ZBH,ZION,ZTS
1990-01-01,8.524683,0.364147,1.906826,NaN,NaN,NaN,NaN,1.335044,NaN,3.8125,...,NaN,4.220179,12.068027,NaN,NaN,NaN,NaN,NaN,2.024005,NaN
1990-01-08,8.485036,0.360612,1.854727,NaN,NaN,NaN,NaN,1.320373,NaN,3.7500,...,NaN,4.030815,11.552299,NaN,NaN,NaN,NaN,NaN,1.988496,NaN
1990-01-15,8.551114,0.353541,1.899146,NaN,NaN,NaN,NaN,1.337989,NaN,3.7500,...,NaN,4.017285,11.603876,NaN,NaN,NaN,NaN,NaN,1.970742,NaN
1990-01-22,8.035671,0.342935,1.804887,NaN,NaN,NaN,NaN,1.411505,NaN,3.6875,...,NaN,3.936128,11.036574,NaN,NaN,NaN,NaN,NaN,1.881970,NaN
1990-01-29,8.260348,0.341438,1.836307,NaN,NaN,NaN,NaN,1.426208,NaN,3.5000,...,NaN,4.030815,11.423365,NaN,NaN,NaN,NaN,NaN,1.917478,NaN


In [47]:
stock_database.dropna(how='all', axis=1, inplace=True)
stock_database.dropna(how='all', axis=0, inplace=True)
#stock_database.head()

In [48]:
output_path='./data/'
stock_database.to_csv(output_path+'stock_database_weekly.csv')